## PERIN

Preprocess the data from the stancer pipeline and bring it into canonical form so it can be used by the sentiment graph parser.

In [1]:
# IMPORTS
import pandas as pd
import json
import os

In [2]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/Perin_Preprocessing", exist_ok=True)

In [3]:
TRAIN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_val.csv"

TRAIN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_train.json"
TEST_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_test.json"
VAL_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_val.json"

# Full phrase or Head mode
FULL_PHRASE = False
USE_FULL_SENTENCE = True

In [4]:
# read in each dataframe
input_paths = [TRAIN_INPUT_PATH, TEST_INPUT_PATH, VAL_INPUT_PATH]
dataframes = []

for ip in input_paths:
    df = pd.read_csv(ip, index_col=0)
    dataframes.append(df)


In [5]:
# inspect a df
dataframes[1]

,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,arg1_head_start,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
doc_id,,,,,,,,,,,,,,,,,,,,,
5e1b48623e9c283116b7f288f1c387c7857694243c68e16a49cf8f048112b600,verfehlten,12,22,verfehlen,Die Piloten,0,11,N,Piloten,4,...,die Landebahn,23,36,N,Landebahn,27,36,neutral,"Predicate(type='neutral', args=(Head(sentence=...","Die Piloten verfehlten die Landebahn , womit e..."
1e8fed5a9d71bf3b3e8a8786c3b1781ea27caabefd113700fae19dd3bc5a89dd,profitieren,25,36,profitieren,Auch diese beiden Länder,0,24,N,Länder,18,...,der griechischen Aufrüstung,41,68,N,Aufrüstung,58,68,neutral,"Predicate(type='neutral', args=(Head(sentence=...",Auch diese beiden Länder profitieren von der g...
1699345e1fe72f368f0aa47abef61ff0497d2a4ad9ddca46ade3138bf009d6dc,stellte,14,21,stellen,Die Kanzlerin,0,13,N,Kanzlerin,4,...,die deutsch-französischen Vorschläge vom Sommer,34,81,N,Vorschläge,60,70,pro,"Predicate(type='pro', args=(Head(sentence=21, ...",Die Kanzlerin stellte sich hinter die deutsch-...
03ed147000d1265383f1ccda802ccd5bab9f48f2f36890db8a131fc5d5bb7101,stellt,107,113,stellen,die Frage,119,129,N,Frage,123,...,Mittwochabend ( SRF 2,184,206,N,Mittwochabend,184,197,pro,"Predicate(type='pro', args=(Head(sentence=2, t...",Der FC Basel hat das Hinspiel des Achtelfinals...
8f4b4ded1a69c2f35a1cc90304f273fc2dafba74819e81ec0fbeb89a40b53fa8,freut,91,96,freuen,Der geneigte Staatsbürger,0,25,N,Staatsbürger,13,...,das Sprachvermögen des Ministers,112,145,N,Sprachvermögen,116,130,pro,"Predicate(type='pro', args=(Head(sentence=6, t...",Der geneigte Staatsbürger teilt derweil natürl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962241259bd10afbaa7a810239601011a70f5564b42c6a3566ae443ee704f9f3,zurückgewiesen,60,74,zurückweisen,Aufsichtsrat und Vorstand,0,25,N,Aufsichtsrat,0,...,das Angebot,33,44,N,Angebot,37,44,con,"Predicate(type='con', args=(Head(sentence=8, t...",Aufsichtsrat und Vorstand hatten das Angebot a...
f3c360118c5631935244ef53fccb3ab2ecbc38f09843460e11a87098388c93d1,verzweifelte,31,43,verzweifeln,Der Nirvana-Sänger Kurt Cobain,0,30,N,Nirvana-Sänger,4,...,unserer Apathie,47,62,N,Apathie,55,62,con,"Predicate(type='con', args=(Head(sentence=62, ...",Der Nirvana-Sänger Kurt Cobain verzweifelte an...
dce5ee9c540914baea0ab70f73c4c7f907a3b17189e482e8e8e64da1e868f761,übergeben,32,41,übergeben,Der Entscheid,0,14,N,Entscheid,4,...,das Geschäft,16,28,N,Geschäft,20,28,neutral,"Predicate(type='neutral', args=(Head(sentence=...","Der Entscheid , das Geschäft zu übergeben , se..."


In [6]:
dataframes[1].rel_type.value_counts()

pro        2752
con        2226
neutral    1755
Name: rel_type, dtype: int64

In [7]:
def transform_to_semeval_ssa_format(df):
    sent_list = []
    sentences = list(set(df.full_sentence_text.to_list()))
    for i, sentence in enumerate(sentences):
        opinions_list = []
        sub_df = df[df["full_sentence_text"] == sentence]
        for row in sub_df.itertuples():
            # skip neutral case, not needed here
            # if row.rel_type == "neutral":
            #    continue

            if FULL_PHRASE:
                # only-NPs
                opinions_list.append({
                    "Source": [[row.arg1], [f"{row.arg1_start}:{row.arg1_end}"]],
                    "Target": [[row.arg2], [f"{row.arg2_start}:{row.arg2_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })
            else:
                # non-NPs
                opinions_list.append({
                    "Source": [[row.arg1_head], [f"{row.arg1_head_start}:{row.arg1_head_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_head_start}:{row.arg2_head_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })

        if len(opinions_list) != 0:
            sent_list.append({
                "sent_id": str(i),
                "text": sentence,
                "opinions": opinions_list
            })
        else:
            continue
    return sent_list

In [8]:
# test the function
data = transform_to_semeval_ssa_format(dataframes[1])

In [9]:
# verify data
data[:1]

[{'sent_id': '0',
  'text': 'Die Schweiz sollte sich hüten , sich der EU zu unterwerfen und das von ihr geforderte Rahmenabkommen zu unterzeichnen .',
  'opinions': [{'Source': [['Schweiz'], ['4:11']],
    'Target': [['EU'], ['41:43']],
    'Polar_expression': [['unterwerfen'], ['47:58']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]}]

In [10]:
FULL_PHRASE

False

In [11]:
x = "Und auch die unabhängigen Verwaltungsräte sowie Konzernleitung und Management stellten sich quer und rebellierten gegen die unerwünschte Übernahme ."
eval('x[' + '26:41' + ']')

'Verwaltungsräte'

## Minimal Sequence Dataset

The idea here is to see whether a pruning approach, that is learning to make the sentence shorter through syntactic dependency reduction would yield an improved result.

This is, would learning to prune help? https://aclanthology.org/D18-1244/

In [12]:
import copy

def convert_ssa_to_minimal_sequence_data(data):
    data = copy.deepcopy(data)
    ret_data = []
    for sent in data:
        opinions = []
        minimal_sent = ""
        for opinion in sent["opinions"]:
            try:
                # print("OsS", opinion["Source"][1][0])
                minimal_sent += eval('sent["text"][' + opinion["Source"][1][0] + ']') + " "
                # print(opinion["Source"])
                opinion["Source"][1][0] = str(minimal_sent.find(opinion["Source"][0][0])) + ":" + str(len(opinion["Source"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Target"][1][0] + ']') + " "
                # print(eval('sent["text"][' + opinion["Target"][1][0] + ']') + " ")
                # print(minimal_sent)
                # print(opinion["Target"])
                opinion["Target"][1][0] = str(minimal_sent.find(opinion["Target"][0][0])) + ":" + str(len(opinion["Target"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Polar_expression"][1][0] + ']') + " "
                opinion["Polar_expression"][1][0] = str(minimal_sent.find(opinion["Polar_expression"][0][0])) + ":" + str(len(opinion["Polar_expression"][0][0]))
                opinions.append(opinion)
            except:
                continue
        if len(opinions) > 0:
            sent["opinions"] = opinions
            # add a dot to every sentence at the end, possibly fixes problem
            sent["text"] = minimal_sent + "."
            ret_data.append(sent)
        else:
            continue
    return ret_data

In [13]:
# data_minimal

In [14]:
output_paths = [TRAIN_OUTPUT_PATH, TEST_OUTPUT_PATH, VAL_OUTPUT_PATH]

for op, df in zip(output_paths, dataframes):
    # call function for each... dev, test, train-json
    data = transform_to_semeval_ssa_format(df)
    # convert to JSON format.
    if not USE_FULL_SENTENCE:
        data = convert_ssa_to_minimal_sequence_data(data)
    with open(op, mode="w") as f:
        json.dump(data, f, ensure_ascii=False)

In [15]:
USE_FULL_SENTENCE

True